## Download abstracts
To fetach article abstracts via https://jeffhuang.com/best_paper_awards/

In [ ]:
from selenium import webdriver

browser = webdriver.Chrome("~/package/chromedriver")

In [ ]:
search = browser.find_element_by_class_name("search-s-facet__button").click()

In [1]:
##

In [2]:
#import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

from time import sleep
from random import randint

In [16]:
from requests.exceptions import MissingSchema
from scholarly import scholarly

In [4]:
work_dir = "/Users/lcx/Documents/machine_learning/projects/text_topic_analysis/"

In [3]:
# Scrape the desired url, only one page

url = 'https://jeffhuang.com/best_paper_awards/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table = soup.find("table")

In [10]:
theads = table.find_all("thead")

### ==== some inspection to understand the html file structure (BEGIN) ====

In [12]:
theads[0]

<thead><td colspan="3"><a name="aaai">AAAI (Artificial Intelligence)</a></td></thead>

In [18]:
theads[0].find("td").get_text()

'AAAI (Artificial Intelligence)'

In [33]:
theads[0].find("td").a

<a name="aaai">AAAI (Artificial Intelligence)</a>

In [13]:
len(theads)

29

In [27]:
table.find("thead")

<thead><td colspan="3"><a name="aaai">AAAI (Artificial Intelligence)</a></td></thead>

In [39]:
table.find("thead").next_sibling.a.get_text()

'WINOGRANDE: An Adversarial Winograd Schema Challenge at Scale'

In [42]:
for thead in theads:
    print(thead.find("td").get_text())
    print(thead.find("td").a["name"])

AAAI (Artificial Intelligence)
aaai
ACL (Natural Language Processing)
acl
CHI (Human-Computer Interaction)
chi
CIKM (Knowledge Management)
cikm
CVPR (Computer Vision)
cvpr
FOCS (Theory)
focs
FSE (Software Engineering)
fse
ICCV (Computer Vision)
iccv
ICML (Machine Learning)
icml
ICSE (Software Engineering)
icse
IJCAI (Artificial Intelligence)
ijcai
INFOCOM (Networking)
infocom
KDD (Data Mining)
kdd
MOBICOM (Mobile)
mobicom
NSDI (Networking)
nsdi
OSDI (Operating Systems)
osdi
PLDI (Programming Languages)
pldi
PODS (Databases)
pods
S&P (Security and Privacy)
s&p
SIGCOMM (Networking)
sigcomm
SIGIR (Information Retrieval)
sigir
SIGMETRICS (Performance)
sigmetrics
SIGMOD (Databases)
sigmod
SODA (Theory)
soda
SOSP (Operating Systems)
sosp
STOC (Theory)
stoc
UIST (User Interface)
uist
VLDB (Databases)
vldb
WWW (World Wide Web)
www


In [47]:
print(table.find("thead").next_sibling.a.has_attr("name"))
print(table.find("thead").next_sibling.a.has_attr("href"))

False
True


In [54]:
thead_test = table.find("thead").find_next_sibling()

In [38]:
print(table.find("thead").next_sibling.prettify())

<tr>
 <td rowspan="1">
  <b>
   2020
  </b>
 </td>
 <td>
  <a href="https://www.semanticscholar.org/paper/8f7133b2e3851b09d659b91e8faa761ec206413f">
   WINOGRANDE: An Adversarial Winograd Schema Challenge at Scale
  </a>
 </td>
 <td>
  Keisuke Sakaguchi, Allen Institute for Artificial Intelligence
  <a href="#" onclick="this.nextSibling.style.display = 'block'; this.style.display = 'none'; return false;">
   ; et al.
  </a>
  <div>
   Ronan Le Bras, Allen Institute for Artificial Intelligence
   <br/>
   Chandra Bhagavatula, Allen Institute for Artificial Intelligence
   <br/>
   Yejin Choi, University of Washington
  </div>
 </td>
</tr>



### ==== some inspection to understand the html file structure (END) ====

### 1. Parse the html file and extract the conference and paper information

["conf_abbr", "conference_name", "year", "name", "author_info", "article_link"]

In [83]:
rows = []

for thead in theads:
    print(thead.find("td").get_text())
    # conference name
    conf_name = thead.find("td").get_text()
    # conference abbreviation
    conf_name_abb = thead.find("td").a["name"]
    
    # get the next sibling
    nextSibling = thead.find_next_sibling();
    
    article_count = 0
    
    # if the next sibling is not None and not the journal name --> it's an article
    while (nextSibling and not nextSibling.a.has_attr("name")):
        
        tds = nextSibling.find_all("td")
        
        # how many papers in that year
        if (tds[0].has_attr("rowspan")): 
            paper_num = int(tds[0]["rowspan"])
            if (paper_num > 6): print("many paper in this year: ", tds[0].b.get_text())
        elif (pre_paper != 0):
            paper_num = int(pre_paper) - 1
        else:
            raise Exception("fail to figure out the number of paper")
            
        
        # year
        try:
            year_str = tds[0].b.get_text()
        except AttributeError as error:
            
            if (paper_num != 0):
                year_str = pre_year_str
                # print("cannot find year, use the year for previous paper")
            else:
                raise Exception("fail to assign a valid year")

        
        try:
            article_link = tds[-2].a["href"]
        except KeyError as error:
            print("don't have attribute name")
        except:
            print("unexpected error")
            raise
        
        article_name = tds[-2].a.get_text()
        # author information
        author_info = tds[-1].get_text("|")
        
        article_count += 1
        pre_paper = paper_num
        pre_year_str = year_str
        
        # print("article: ", article_count, " year: ", year_str, " name: ", article_name)
        
        # row add
        rows.append([conf_name_abb, conf_name, year_str, article_name, author_info, article_link])
        #
        nextSibling = nextSibling.find_next_sibling();
    
    print("article found: ", article_count)

AAAI (Artificial Intelligence)
article found:  39
ACL (Natural Language Processing)
article found:  25
CHI (Human-Computer Interaction)
many paper in this year:  2020
many paper in this year:  2019
many paper in this year:  2018
many paper in this year:  2017
many paper in this year:  2016
many paper in this year:  2015
many paper in this year:  2014
many paper in this year:  2013
many paper in this year:  2012
many paper in this year:  2011
many paper in this year:  2010
many paper in this year:  2009
many paper in this year:  2008
article found:  252
CIKM (Knowledge Management)
article found:  17
CVPR (Computer Vision)
article found:  21
FOCS (Theory)
article found:  34
FSE (Software Engineering)
many paper in this year:  2017
many paper in this year:  2016
many paper in this year:  2015
article found:  69
ICCV (Computer Vision)
article found:  17
ICML (Machine Learning)
article found:  22
ICSE (Software Engineering)
many paper in this year:  2019
many paper in this year:  2018
many 

In [84]:
df=pd.DataFrame(rows,columns=["conf_abbr", "conference_name", "year", "name", "author_info", "article_link"])

In [85]:
df.shape

(1072, 6)

In [86]:
df.head(5)

,conf_abbr,conference_name,year,name,author_info,article_link
0,aaai,AAAI (Artificial Intelligence),2020,WINOGRANDE: An Adversarial Winograd Schema Cha...,"Keisuke Sakaguchi, Allen Institute for Artific...",https://www.semanticscholar.org/paper/8f7133b2...
1,aaai,AAAI (Artificial Intelligence),2019,How to Combine Tree-Search Methods in Reinforc...,"Yonathan Efroni, Technion – Israel Institute o...",https://www.semanticscholar.org/paper/How-to-C...
2,aaai,AAAI (Artificial Intelligence),2018,Memory-Augmented Monte Carlo Tree Search,"Chenjun Xiao, University of Alberta|; et al.|J...",https://www.semanticscholar.org/paper/2d7fb18f...
3,aaai,AAAI (Artificial Intelligence),2017,Label-Free Supervision of Neural Networks with...,"Russell Stewart & Stefano Ermon, Stanford Univ...",https://www.semanticscholar.org/paper/2ee62982...
4,aaai,AAAI (Artificial Intelligence),2016,Bidirectional Search That Is Guaranteed to Mee...,"Robert C. Holte, University of Alberta|; et al...",https://www.semanticscholar.org/paper/d4c2696a...


In [87]:
df.to_csv("../data/Best_Paper_Awards_in_Computer_Science_since_1996_raw.csv",header=True)

## 2. filter the dataset to reduce the size

In [6]:
#
df_all_article_raw = pd.read_csv("../data/Best_Paper_Awards_in_Computer_Science_since_1996_raw.csv", index_col=0)

In [7]:
df_all_article_raw.head(5)

,conf_abbr,conference_name,year,name,author_info,article_link
0,aaai,AAAI (Artificial Intelligence),2020,WINOGRANDE: An Adversarial Winograd Schema Cha...,"Keisuke Sakaguchi, Allen Institute for Artific...",https://www.semanticscholar.org/paper/8f7133b2...
1,aaai,AAAI (Artificial Intelligence),2019,How to Combine Tree-Search Methods in Reinforc...,"Yonathan Efroni, Technion – Israel Institute o...",https://www.semanticscholar.org/paper/How-to-C...
2,aaai,AAAI (Artificial Intelligence),2018,Memory-Augmented Monte Carlo Tree Search,"Chenjun Xiao, University of Alberta|; et al.|J...",https://www.semanticscholar.org/paper/2d7fb18f...
3,aaai,AAAI (Artificial Intelligence),2017,Label-Free Supervision of Neural Networks with...,"Russell Stewart & Stefano Ermon, Stanford Univ...",https://www.semanticscholar.org/paper/2ee62982...
4,aaai,AAAI (Artificial Intelligence),2016,Bidirectional Search That Is Guaranteed to Mee...,"Robert C. Holte, University of Alberta|; et al...",https://www.semanticscholar.org/paper/d4c2696a...


In [8]:
df_index_lookup = pd.read_csv("../data/top_conference_ML_AI.csv", index_col=0)

In [9]:
df_index_lookup.head(5)

,h5_index,Name,URL
0,240,CVPR : IEEE/CVF Conference on Computer Vision ...,http://cvpr2020.thecvf.com/
1,169,NeurIPS : Neural Information Processing System...,https://nips.cc/Conferences/2020/CallForPapers
2,137,ECCV : European Conference on Computer Vision,https://eccv2020.eu/
3,135,ICML : International Conference on Machine Lea...,https://icml.cc/Conferences/2020
4,129,ICCV : IEEE/CVF International Conference on Co...,http://iccv2021.thecvf.com/home


In [28]:
df_index_lookup["Abbre"] = df_index_lookup["Name"].apply(lambda x: x.split(":", 1)[0].strip())

In [29]:
df_index_lookup.head(5)

,h5_index,Name,URL,Abbre
0,240,CVPR : IEEE/CVF Conference on Computer Vision ...,http://cvpr2020.thecvf.com/,CVPR
1,169,NeurIPS : Neural Information Processing System...,https://nips.cc/Conferences/2020/CallForPapers,NeurIPS
2,137,ECCV : European Conference on Computer Vision,https://eccv2020.eu/,ECCV
3,135,ICML : International Conference on Machine Lea...,https://icml.cc/Conferences/2020,ICML
4,129,ICCV : IEEE/CVF International Conference on Co...,http://iccv2021.thecvf.com/home,ICCV


In [32]:
df_index_lookup.to_csv("../data/top_conference_ML_AI_Abbre.csv", header=True)

### 2.1. decide what conference and years we would like to save

In [47]:
top_k = 15

df_select_top = pd.DataFrame(columns=df_all_article_raw.columns)
total_num = 0

collect_conf = []
miss_conf = []

for i in np.arange(top_k):
    conf_abbre = df_index_lookup.iloc[i]["Abbre"]
    conf_abbre = conf_abbre.lower()
    collect_conf.append(conf_abbre)
    
    df_tmp = df_all_article_raw[df_all_article_raw["conf_abbr"] == conf_abbre]
    num_paper_found = df_tmp.shape[0]
    
    
    if (num_paper_found == 0):
        print("no paper found for: ", conf_abbre)
        miss_conf.append(conf_abbre)
        continue
    
    start_year = df_tmp.iloc[0]["year"]; end_year = df_tmp.iloc[-1]["year"] 
    print("conference: ", conf_abbre, ", ", start_year, " - ", end_year, " numbers: ", df_tmp.shape[0])
    total_num += df_tmp.shape[0]
    
    df_select_top = pd.concat([df_select_top, df_tmp], axis=0, ignore_index=True)

print("total number of papers collected:", total_num)

paper_conf_info = {"collections": collect_conf, "missing": miss_conf}

conference:  cvpr ,  2019  -  2000  numbers:  21
no paper found for:  neurips
no paper found for:  eccv
conference:  icml ,  2019  -  1999  numbers:  22
conference:  iccv ,  2019  -  1998  numbers:  17
conference:  acl ,  2019  -  2001  numbers:  25
conference:  aaai ,  2020  -  1996  numbers:  39
no paper found for:  emnlp
no paper found for:  sigkdd
conference:  ijcai ,  2019  -  1997  numbers:  33
conference:  sigmod ,  2020  -  1994  numbers:  29
no paper found for:  interspeech
no paper found for:  iros
conference:  sigir ,  2019  -  1996  numbers:  24
no paper found for:  isit
total number of papers collected: 210


In [44]:
df_select_top.head(5)

,conf_abbr,conference_name,year,name,author_info,article_link
0,cvpr,CVPR (Computer Vision),2019,A Theory of Fermat Paths for Non-Line-of-Sight...,"Shumian Xin, Carnegie Mellon University|; et a...",semanticscholar.org/paper/5809decf0a47a8a6be5c...
1,cvpr,CVPR (Computer Vision),2018,Taskonomy: Disentangling Task Transfer Learning,"Amir R. Zamir, Stanford University|; et al.|Al...",https://www.semanticscholar.org/paper/2fe2cfd9...
2,cvpr,CVPR (Computer Vision),2017,Densely Connected Convolutional Networks,"Zhuang Liu, Tsinghua University|; et al.|Gao H...",https://www.semanticscholar.org/paper/5694e462...
3,cvpr,CVPR (Computer Vision),2017,Learning from Simulated and Unsupervised Image...,"Ashish Shrivastava, Apple Inc.|; et al.|Tomas ...",https://www.semanticscholar.org/paper/68cb9fce...
4,cvpr,CVPR (Computer Vision),2016,Deep Residual Learning for Image Recognition,"Kaiming He, Microsoft Research|; et al.|Xiangy...",https://www.semanticscholar.org/paper/29c808b3...


In [45]:
df_select_top.to_csv("../data/top_k_conf_paper.csv", header=True)

In [46]:
miss_conf

['neurips', 'eccv', 'emnlp', 'sigkdd', 'interspeech', 'iros', 'isit']

In [49]:
print(paper_conf_info)

{'collections': ['cvpr', 'neurips', 'eccv', 'icml', 'iccv', 'acl', 'aaai', 'emnlp', 'sigkdd', 'ijcai', 'sigmod', 'interspeech', 'iros', 'sigir', 'isit'], 'missing': ['neurips', 'eccv', 'emnlp', 'sigkdd', 'interspeech', 'iros', 'isit']}


In [51]:
with open("../data/conference_collections.json", "w") as fp:
    json.dump(paper_conf_info, fp)

In [53]:
json.dumps(paper_conf_info, indent=4)

'{\n    "collections": [\n        "cvpr",\n        "neurips",\n        "eccv",\n        "icml",\n        "iccv",\n        "acl",\n        "aaai",\n        "emnlp",\n        "sigkdd",\n        "ijcai",\n        "sigmod",\n        "interspeech",\n        "iros",\n        "sigir",\n        "isit"\n    ],\n    "missing": [\n        "neurips",\n        "eccv",\n        "emnlp",\n        "sigkdd",\n        "interspeech",\n        "iros",\n        "isit"\n    ]\n}'

In [54]:
paper_conf_info

{'collections': ['cvpr',
  'neurips',
  'eccv',
  'icml',
  'iccv',
  'acl',
  'aaai',
  'emnlp',
  'sigkdd',
  'ijcai',
  'sigmod',
  'interspeech',
  'iros',
  'sigir',
  'isit'],
 'missing': ['neurips',
  'eccv',
  'emnlp',
  'sigkdd',
  'interspeech',
  'iros',
  'isit']}

## 3. use the url to get the abstract

In [4]:
df_select = pd.read_csv("../data/top_k_conf_paper.csv", index_col=0)

In [5]:
df_select.head()

,conf_abbr,conference_name,year,name,author_info,article_link
0,cvpr,CVPR (Computer Vision),2019,A Theory of Fermat Paths for Non-Line-of-Sight...,"Shumian Xin, Carnegie Mellon University|; et a...",semanticscholar.org/paper/5809decf0a47a8a6be5c...
1,cvpr,CVPR (Computer Vision),2018,Taskonomy: Disentangling Task Transfer Learning,"Amir R. Zamir, Stanford University|; et al.|Al...",https://www.semanticscholar.org/paper/2fe2cfd9...
2,cvpr,CVPR (Computer Vision),2017,Densely Connected Convolutional Networks,"Zhuang Liu, Tsinghua University|; et al.|Gao H...",https://www.semanticscholar.org/paper/5694e462...
3,cvpr,CVPR (Computer Vision),2017,Learning from Simulated and Unsupervised Image...,"Ashish Shrivastava, Apple Inc.|; et al.|Tomas ...",https://www.semanticscholar.org/paper/68cb9fce...
4,cvpr,CVPR (Computer Vision),2016,Deep Residual Learning for Image Recognition,"Kaiming He, Microsoft Research|; et al.|Xiangy...",https://www.semanticscholar.org/paper/29c808b3...


### 3.1 figure out the html structure

In [6]:
art1_url = df_select.loc[0, "article_link"]
print(art1_url)

semanticscholar.org/paper/5809decf0a47a8a6be5c0525cd9f92e1e7d4c75f


In [8]:
from requests.exceptions import MissingSchema

try:
    url = art1_url
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
except MissingSchema:
    url = "https://www." + art1_url
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   [PDF] A Theory of Fermat Paths for Non-Line-Of-Sight Shape Reconstruction | Semantic Scholar
  </title>
  <meta content="noarchive" name="robots"/>
  <!-- We have a public API for this page. Check out http://api.semanticscholar.org/ -->
  <link href="https://www.semanticscholar.org/paper/A-Theory-of-Fermat-Paths-for-Non-Line-Of-Sight-Xin-Nousias/5809decf0a47a8a6be5c0525cd9f92e1e7d4c75f" rel="canonical"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta charset="utf-8"/>
  <meta content="af5b3052e622644aa4e48ea4831c00ccee5bcf68" name="s2-ui-version"/>
  <meta content="We present a novel theory of Fermat paths of light between a known visible scene and an unknown object not in the line of sight of a transient camera. These light paths either obey specular reflection or are reflected by the object's boundary, and hence encode the shape of the hidden object. We prove that Fermat paths correspond to disconti

In [14]:
abstract = soup.find("meta", {"name": "description"})

In [15]:
abstract["content"]

"We present a novel theory of Fermat paths of light between a known visible scene and an unknown object not in the line of sight of a transient camera. These light paths either obey specular reflection or are reflected by the object's boundary, and hence encode the shape of the hidden object. We prove that Fermat paths correspond to discontinuities in the transient measurements. We then derive a novel constraint that relates the spatial derivatives of the path lengths at these discontinuities to the surface normal. Based on this theory, we present an algorithm, called Fermat Flow, to estimate the shape of the non-line-of-sight object. Our method allows, for the first time, accurate shape recovery of complex objects, ranging from diffuse to specular, that are hidden around the corner as well as hidden behind a diffuser. Finally, our approach is agnostic to the particular technology used for transient imaging. As such, we demonstrate mm-scale shape recovery from pico-second scale transie

### 3.2 extract all

In [16]:
nums_paper = df_select.shape[0]
print(nums_paper)

210


In [21]:
df_select["abstract"] = ""

num_start = 0
no_found = []

for i in  np.arange(num_start, nums_paper):
    
    art1_url = df_select.loc[i, "article_link"]
    
    sleep(2)
    
    try:
        url = art1_url
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    except MissingSchema:
        url = "https://www." + art1_url
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
    
    try:    
        abstract = soup.find("meta", {"name": "description"})
        df_select.loc[i, "abstract"] = abstract["content"]
        # TODO: also update the tile, because the paper title from previous can be truncated
    except:
        print("failed at i = ", i)
        no_found.append(i)
    

failed at i =  10
failed at i =  92
failed at i =  98


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


failed at i =  109
failed at i =  124
failed at i =  157
failed at i =  158
failed at i =  160
failed at i =  183
failed at i =  185
failed at i =  206


**what we have missed**

In [22]:
df_select[df_select["abstract"] == ""]

,conf_abbr,conference_name,year,name,author_info,article_link,abstract
10,cvpr,CVPR (Computer Vision),2010,Efficient Computation of Robust Low-Rank Matri...,"Anders Eriksson & Anton va den Hendel, Univers...",http://scholar.google.com/scholar?as_q=&num=10...,
92,aaai,AAAI (Artificial Intelligence),2013,SMILe: Shuffled Multiple-Instance Learning,"Gary Doran & Soumya Ray, Case Western Reserve ...",http://scholar.google.com/scholar?as_q=&num=10...,
98,aaai,AAAI (Artificial Intelligence),2010,How Incomplete Is Your Semantic Web Reasoner? ...,"Giorgos Stoilos, University of Oxford|; et al....",http://scholar.google.com/scholar?as_q=&num=10...,
109,aaai,AAAI (Artificial Intelligence),2000,Statistics-Based Summarization -- Step One: Se...,"Kevin Knight & Daniel Marcu, University of Sou...",https://www.aaai.org/Papers/AAAI/2000/AAAI00-1...,
124,ijcai,IJCAI (Artificial Intelligence),2019,Clause Elimination for SAT and QSAT,"Marijn Heule, University of Texas at Austin|; ...",http://scholar.google.com/scholar?as_q=&num=10...,
157,sigmod,SIGMOD (Databases),2020,Pump Up the Volume: Processing Large Data on G...,"Clemens Lutz, DFKI GmbH|; et al.|Sebastian Bre...",http://scholar.google.com/scholar?as_q=&num=10...,
158,sigmod,SIGMOD (Databases),2020,ShapeSearch: A Flexible and Efficient System f...,"Tarique Siddiqui, University of Illinois at Ur...",http://scholar.google.com/scholar?as_q=&num=10...,
160,sigmod,SIGMOD (Databases),2018,SuRF: Practical Range Query Filtering with Fas...,"Huanchen Zhang, Carnegie Mellon University|; e...",http://scholar.google.com/scholar?as_q=&num=10...,
183,sigmod,SIGMOD (Databases),1996,Fast Subsequence Matching in Time-Series Datab...,"Christos Faloutsos, University of Maryland at ...",http://scholar.google.com/scholar?as_q=&num=10...,
185,sigmod,SIGMOD (Databases),1994,AlphaSort: A RISC Machine Sort,"Chris Nyberg, Digital Equipment Corporation|; ...",http://scholar.google.com/scholar?as_q=&num=10...,


In [30]:
print(no_found)

[10, 92, 98, 109, 124, 157, 158, 160, 183, 185, 206]


**do some quick check**

In [25]:
print("paper:")
print(df_select.loc[0, "name"])
print("\nAbstract:")
print(df_select.loc[0, "abstract"])

paper:
A Theory of Fermat Paths for Non-Line-of-Sight Shape Reconstruction

Abstract:
We present a novel theory of Fermat paths of light between a known visible scene and an unknown object not in the line of sight of a transient camera. These light paths either obey specular reflection or are reflected by the object's boundary, and hence encode the shape of the hidden object. We prove that Fermat paths correspond to discontinuities in the transient measurements. We then derive a novel constraint that relates the spatial derivatives of the path lengths at these discontinuities to the surface normal. Based on this theory, we present an algorithm, called Fermat Flow, to estimate the shape of the non-line-of-sight object. Our method allows, for the first time, accurate shape recovery of complex objects, ranging from diffuse to specular, that are hidden around the corner as well as hidden behind a diffuser. Finally, our approach is agnostic to the particular technology used for transient im

**save the snapshot of the data**

In [29]:
df_select.to_csv(work_dir + "data/top_k_conf_paper_with_abstract.csv", header=True)

### 3.3 handling some missing abstract

**Note: seems like scholarly is ofen blocked by google, so we just save the articles with available abstracts at this point**

In [31]:
df_select[df_select["abstract"].notnull()].to_csv(work_dir + "data/top_k_conf_paper_with_abstract_filter.csv",
                                                  index=False)

=========================

In [6]:
df_select = pd.read_csv(work_dir + "data/top_k_conf_paper_with_abstract.csv", index_col=0)

In [7]:
df_select.head(3)

,conf_abbr,conference_name,year,name,author_info,article_link,abstract
0,cvpr,CVPR (Computer Vision),2019,A Theory of Fermat Paths for Non-Line-of-Sight...,"Shumian Xin, Carnegie Mellon University|; et a...",semanticscholar.org/paper/5809decf0a47a8a6be5c...,We present a novel theory of Fermat paths of l...
1,cvpr,CVPR (Computer Vision),2018,Taskonomy: Disentangling Task Transfer Learning,"Amir R. Zamir, Stanford University|; et al.|Al...",https://www.semanticscholar.org/paper/2fe2cfd9...,"Do visual tasks have a relationship, or are th..."
2,cvpr,CVPR (Computer Vision),2017,Densely Connected Convolutional Networks,"Zhuang Liu, Tsinghua University|; et al.|Gao H...",https://www.semanticscholar.org/paper/5694e462...,Recent work has shown that convolutional netwo...


In [12]:
# using google scholar
to_fix_index = df_select[df_select["abstract"].isnull()].index
print(len(to_fix_index))

11


In [23]:
print("# of missing abstracts: ", df_select["abstract"].isnull().sum())
print("# of non-missing abstracts: ", df_select["abstract"].notnull().sum())

# of missing abstracts:  11
# of non-missing abstracts:  199


In [13]:
print(to_fix_index)

Int64Index([10, 92, 98, 109, 124, 157, 158, 160, 183, 185, 206], dtype='int64')


In [17]:
for idx in [10, 92]:
    article_name = df_select.loc[idx, "name"]
    authors = df_select.loc[idx, "author_info"]
    year = df_select.loc[idx, "year"]
    print(40*"=", "\nidx = ", idx)
    print("paper name: \n", article_name, "\n author info:\n", authors, "\n year: ", year)
    
    try:
        search_query = scholarly.search_pubs(article_name)
        search_result = next(search_query)
        search_result_dict = search_result.__dict__
        
        article_name_sch = search_result_dict["bib"]["title"]
        authors_sch = search_result_dict["bib"]["author"]
        year_sch = int(search_result_dict["bib"]["year"])
        
        print("scholar query result: ")
        print("paper name: \n", article_name_sch, "\n author info:\n", authors_sch, "\n year: ", year_sch)

        if (year == year_sch):

            abstract = search_result_dict["bib"]["abstract"]
            df_select.loc[i, "abstract"] = abstract
            print("char count: ", len(abstract), " word count: ", len(abastract.split()))
        else:
            print("year doesn't match")

    except:
        print("\n\nscholar query failed")
        raise
        
    sleep(2)
        

idx =  10
paper name: 
 Efficient Computation of Robust Low-Rank Matrix Approximations in the Presence of Missing Data usi... 
 author info:
 Anders Eriksson & Anton va den Hendel, University of Adelaide 
 year:  2010
/n/nscholar query failed
idx =  92
paper name: 
 SMILe: Shuffled Multiple-Instance Learning 
 author info:
 Gary Doran & Soumya Ray, Case Western Reserve University 
 year:  2013
/n/nscholar query failed


### === explore scholarly, to delete

In [24]:
df_1 = df_select.loc[10, :]

In [25]:
df_1

conf_abbr                                                       cvpr
conference_name                               CVPR (Computer Vision)
year                                                            2010
name               Efficient Computation of Robust Low-Rank Matri...
author_info        Anders Eriksson & Anton va den Hendel, Univers...
article_link       http://scholar.google.com/scholar?as_q=&num=10...
abstract                                                         NaN
Name: 10, dtype: object

In [26]:
print(df_1["name"])

Efficient Computation of Robust Low-Rank Matrix Approximations in the Presence of Missing Data usi...


In [ ]:
search_query = scholarly.search_pubs(df_1["name"])

In [46]:
search_result = next(search_query)

In [50]:
search_result

{'bib': {'abstract': 'The calculation of a low-rank approximation of a matrix '
                     'is a fundamental operation in many computer vision '
                     'applications. The workhorse of this class of problems '
                     'has long been the Singular Value Decomposition. However, '
                     'in the presence of missing data and outliers this method '
                     'is not applicable, and unfortunately, this is often the '
                     'case in practice. In this paper we present a method for '
                     'calculating the low-rank factorization of a matrix which '
                     'minimizes the L 1 norm in the presence of missing data. '
                     'Our approach represents a generalization the',
         'author': ['A Eriksson', 'A Van Den Hengel'],
         'author_id': [None, 'nMGZ2ZQAAAAJ'],
         'cites': '205',
         'eprint': 'https://eprints.qut.edu.au/108200/1/108200.pdf',
         'gsrank': '

In [51]:
type(search_result)

scholarly.publication.Publication

In [58]:
search_result_dict = search_result.__dict__

In [59]:
search_result_dict["bib"]["abstract"]

'The calculation of a low-rank approximation of a matrix is a fundamental operation in many computer vision applications. The workhorse of this class of problems has long been the Singular Value Decomposition. However, in the presence of missing data and outliers this method is not applicable, and unfortunately, this is often the case in practice. In this paper we present a method for calculating the low-rank factorization of a matrix which minimizes the L 1 norm in the presence of missing data. Our approach represents a generalization the'

In [65]:
len(search_result_dict["bib"]["abstract"])

544

In [68]:
len(search_result_dict["bib"]["abstract"].split())

88